In [1]:
import torch
import numpy as np
import wandb
import pandas as pd

In [2]:
PROJECT_NAME = 'bm3d_vs_superonn'

In [4]:
api = wandb.Api()
runs = api.runs("jmalik/"+PROJECT_NAME)
summary_list = [] 
config_list = [] 
name_list = [] 
id_list = []
for run in runs: 
    summary_list.append(run.summary._json_dict) 
    config_list.append({k:v for k,v in run.config.items() if not k.startswith('_')}) 
    name_list.append(run.name)
    id_list.append(run.id)       
summary_df = pd.DataFrame.from_records(summary_list) 
config_df = pd.DataFrame.from_records(config_list) 
name_df = pd.DataFrame({'name': name_list})
id_df = pd.DataFrame({'ID': id_list})  
df = pd.concat([id_df,name_df, config_df,summary_df], axis=1)
splitted = df['name'].str.split('_',n=1,expand=True)
df['Run Name'] = splitted[0]
df['Version'] = splitted[1]
df = df.drop(columns=['name'])
idx = df.groupby(['Run Name'])['top_val_psnr'].transform(max) == df['top_val_psnr']
df = df[idx]

# remove crashed runs
df = df.loc[df['epoch']==99]

In [5]:
df

,ID,q,clip,sigma,max_shifts,model_name,num_layers,num_neurons,train_ratio,dataset_name,...,train_loss,top_val_psnr,McM,Kodak,CBSD68,_runtime,trainer/global_step,global_step,Run Name,Version
0,4mqdhybd,1,True,0,"[0, 0, 0]",ridnet,0,0,0.95,sidd,...,319.378265,41.284206,30.812317,31.107117,30.256775,129369.0,NaN,NaN,9dcfjy,0
1,3srmqe1b,1,True,0,"[0, 0, 0]",ridnet,0,0,0.95,sidd,...,316.921722,41.304935,30.141701,29.892746,29.284250,181342.0,NaN,NaN,jdgea6,0
2,1brdc1mg,1,True,0,"[0, 10, 10]",NaN,17,1024,0.95,sidd,...,3072.462646,34.058853,NaN,NaN,NaN,46024.0,60799.0,NaN,7wrwdv,0
4,26i8ov3y,7,True,0,"[0, 0, 0]",NaN,4,256,0.95,sidd,...,1360.492188,37.980061,26.747463,26.891420,25.973528,46397.0,NaN,60799.0,whzdfg,0
5,26pemhmz,5,True,0,"[0, 0, 0]",NaN,4,256,0.95,sidd,...,1376.509033,38.191158,27.118246,27.469292,26.870987,35008.0,NaN,60799.0,zupt7e,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1qsbj8t9,5,True,90,NaN,NaN,4,256,0.95,dncnn,...,622.264038,23.619362,24.722420,24.575203,23.579559,10830.0,NaN,353799.0,lxv3ql,0
210,3sc1pee9,3,True,60,NaN,NaN,4,256,0.95,dncnn,...,435.218781,25.138136,26.623148,26.128340,25.109344,8620.0,NaN,353799.0,vjw7nm,0
211,1u9bp41f,7,True,60,NaN,NaN,4,256,0.95,dncnn,...,432.140564,25.169771,26.557747,26.130699,25.076212,13032.0,NaN,353799.0,l7widm,0
212,2m5pry5c,5,True,60,NaN,NaN,4,256,0.95,dncnn,...,435.936981,25.146667,26.255165,25.924934,24.911688,10799.0,NaN,353799.0,1pltex,0


In [16]:
df['max_shifts'] = '[0,0,0]'

In [17]:
df_synthetic_denoising = df.loc[df.dataset_name=='dncnn']
df_rw_denoising = df.loc[df.dataset_name=='sidd']

In [8]:
df_synthetic_denoising_deep = df_synthetic_denoising.loc[df.num_neurons==1024]
df_synthetic_denoising_deep.to_pickle(
    'df_dncnn.pkl'
)

In [7]:
df_synthetic_denoising_compact = df_synthetic_denoising.loc[df.num_neurons<1024]

In [10]:
# download runs, load model and replace dummy max shifts with actual shifts
import paramiko
import os

def get_checkpoint_from_remote(sftp,run_name):
    # Generate paths
    remotepath = '/scratch/zhangh/malik/BM3D_VS_SELFONN/{0}/{1}/checkpoints/'.format(PROJECT_NAME,run_name)
    localpath = '{0}/{1}/checkpoints/'.format(PROJECT_NAME,run_name)
    
    # Create localpath if not exists
    if not os.path.exists(localpath): 
        os.makedirs(localpath)
        print('Created local directory {0}'.format(localpath))
    
    # Get latest checkpoint and append to remotepath
    for filename in sftp.listdir(remotepath): break
    remotepath += filename
    localpath += filename
    
    # Skip if localfile exists
    if os.path.exists(localpath): 
        print(f"Checkpoint {localpath} already exists. Skipping..")
        return;

    # Transfer checkpoint from remote to local
    sftp.get(remotepath, localpath)
    


In [11]:
# download all checkpoints
# Initiate connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('puhti.csc.fi', username="malik", password="TuniPassword2021")
sftp = ssh.open_sftp()
print('Connection open..')

# Download checkpoints
#for index,row in df_synthetic_denoising_compact.iterrows(): get_checkpoint_from_remote(sftp,row.ID)
for index,row in df_synthetic_denoising_deep.iterrows(): get_checkpoint_from_remote(sftp,row.ID)

# Close connection
sftp.close()
ssh.close()
print('Connection closed..')
    

Connection open..
Created local directory bm3d_vs_superonn/3g4s8xs3/checkpoints/
Created local directory bm3d_vs_superonn/dzvequfx/checkpoints/
Created local directory bm3d_vs_superonn/3hrokin1/checkpoints/
Connection closed..


In [10]:
%load_ext autoreload
%autoreload 1

In [11]:
import glob
%aimport utils.denoiser
%aimport utils.models

# drop max_shifts
if 'max_shifts' in df_synthetic_denoising_compact: df_synthetic_denoising_compact.drop(['max_shifts'],inplace=True,axis=1)
df['max_shifts'] = np.nan

# load checkpoints and update max_shifts
ckpt_path = r"{project}//{id}//checkpoints//*.ckpt"
for index,row in df_synthetic_denoising_compact.iterrows():
    # format path for checkpoint
    ckpt_path_now = glob.glob( ckpt_path.format(project=PROJECT_NAME,id=row.ID) )[0]
    
    # load checkpoint
    model = utils.models.get_model('superonn')
    dn = utils.denoiser.Denoiser.load_from_checkpoint(ckpt_path_now,model=model,strict=False,map_location='cpu')

    # get shifts from model
    model_now = dn.model.model
    assert model_now[2].max_shift == model_now[4].max_shift, "Shifts do not match"
    df_synthetic_denoising_compact.loc[df_synthetic_denoising_compact.ID==row.ID,'max_shifts'] = f"[{model_now[0].max_shift},{model_now[2].max_shift},{model_now[6].max_shift}]"

C:\Users\malik\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\malik\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\malik\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [12]:
df_synthetic_denoising_compact

,ID,q,clip,sigma,model_name,num_layers,num_neurons,train_ratio,dataset_name,num_channels,...,_timestamp,train_loss,top_val_psnr,epoch,CBSD68,trainer/global_step,global_step,Run Name,Version,max_shifts
13,1rjr0f27,7,True,30,NaN,4,512,0.95,dncnn,1,...,1.620126e+09,210.293518,28.278606,99.0,28.066206,NaN,353799.0,bmnviu,0,"[1,5,1]"
14,tnjjfjpn,5,True,30,NaN,4,512,0.95,dncnn,1,...,1.620120e+09,211.740295,28.240484,99.0,28.103859,NaN,353799.0,wyjs64,0,"[1,5,1]"
15,3p3llvdc,3,True,90,NaN,4,512,0.95,dncnn,1,...,1.620111e+09,630.040283,23.523420,99.0,23.586006,NaN,353799.0,4m96ac,0,"[1,5,1]"
16,1vnzy85r,3,True,60,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,426.114929,25.231638,99.0,25.168697,NaN,353799.0,w5rkeg,0,"[1,5,1]"
17,md12pcjd,3,True,30,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,217.389618,28.126810,99.0,28.016939,NaN,353799.0,xxd4sh,0,"[1,5,1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1qsbj8t9,5,True,90,NaN,4,256,0.95,dncnn,1,...,1.617621e+09,622.264038,23.619362,99.0,23.579559,NaN,353799.0,lxv3ql,0,"[0,10,10]"
210,3sc1pee9,3,True,60,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,435.218781,25.138136,99.0,25.109344,NaN,353799.0,vjw7nm,0,"[0,10,10]"
211,1u9bp41f,7,True,60,NaN,4,256,0.95,dncnn,1,...,1.617623e+09,432.140564,25.169771,99.0,25.076212,NaN,353799.0,l7widm,0,"[0,10,10]"
212,2m5pry5c,5,True,60,NaN,4,256,0.95,dncnn,1,...,1.617620e+09,435.936981,25.146667,99.0,24.911688,NaN,353799.0,1pltex,0,"[0,10,10]"


In [13]:
# KEEP ONLY ONE COPY OF EACH EXPERIMENT, BASED ON BEST VALIDATION PERFORMANCE
idx = np.where(df_synthetic_denoising_compact.top_val_psnr==df_synthetic_denoising_compact.groupby(['sigma','num_neurons','q','max_shifts'])['top_val_psnr'].transform(max))
df_synthetic_denoising_compact_unique = df_synthetic_denoising_compact.iloc[idx]
df_synthetic_denoising_compact_unique

,ID,q,clip,sigma,model_name,num_layers,num_neurons,train_ratio,dataset_name,num_channels,...,_timestamp,train_loss,top_val_psnr,epoch,CBSD68,trainer/global_step,global_step,Run Name,Version,max_shifts
13,1rjr0f27,7,True,30,NaN,4,512,0.95,dncnn,1,...,1.620126e+09,210.293518,28.278606,99.0,28.066206,NaN,353799.0,bmnviu,0,"[1,5,1]"
14,tnjjfjpn,5,True,30,NaN,4,512,0.95,dncnn,1,...,1.620120e+09,211.740295,28.240484,99.0,28.103859,NaN,353799.0,wyjs64,0,"[1,5,1]"
15,3p3llvdc,3,True,90,NaN,4,512,0.95,dncnn,1,...,1.620111e+09,630.040283,23.523420,99.0,23.586006,NaN,353799.0,4m96ac,0,"[1,5,1]"
16,1vnzy85r,3,True,60,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,426.114929,25.231638,99.0,25.168697,NaN,353799.0,w5rkeg,0,"[1,5,1]"
17,md12pcjd,3,True,30,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,217.389618,28.126810,99.0,28.016939,NaN,353799.0,xxd4sh,0,"[1,5,1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2rs65sx5,3,True,90,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,624.264587,23.592827,99.0,23.636141,NaN,353799.0,h5cfb2,0,"[0,10,10]"
209,1qsbj8t9,5,True,90,NaN,4,256,0.95,dncnn,1,...,1.617621e+09,622.264038,23.619362,99.0,23.579559,NaN,353799.0,lxv3ql,0,"[0,10,10]"
210,3sc1pee9,3,True,60,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,435.218781,25.138136,99.0,25.109344,NaN,353799.0,vjw7nm,0,"[0,10,10]"
211,1u9bp41f,7,True,60,NaN,4,256,0.95,dncnn,1,...,1.617623e+09,432.140564,25.169771,99.0,25.076212,NaN,353799.0,l7widm,0,"[0,10,10]"


In [22]:
df_synthetic_denoising_compact_unique.to_pickle(
    'df_all.pkl'
)

In [14]:
# keep only self-onn networks
df_synthetic_denoising_compact_unique_selfonn = df_synthetic_denoising_compact_unique.iloc[
    np.where(df_synthetic_denoising_compact_unique.q != 1)
]
df_synthetic_denoising_compact_unique_selfonn

,ID,q,clip,sigma,model_name,num_layers,num_neurons,train_ratio,dataset_name,num_channels,...,_timestamp,train_loss,top_val_psnr,epoch,CBSD68,trainer/global_step,global_step,Run Name,Version,max_shifts
13,1rjr0f27,7,True,30,NaN,4,512,0.95,dncnn,1,...,1.620126e+09,210.293518,28.278606,99.0,28.066206,NaN,353799.0,bmnviu,0,"[1,5,1]"
14,tnjjfjpn,5,True,30,NaN,4,512,0.95,dncnn,1,...,1.620120e+09,211.740295,28.240484,99.0,28.103859,NaN,353799.0,wyjs64,0,"[1,5,1]"
15,3p3llvdc,3,True,90,NaN,4,512,0.95,dncnn,1,...,1.620111e+09,630.040283,23.523420,99.0,23.586006,NaN,353799.0,4m96ac,0,"[1,5,1]"
16,1vnzy85r,3,True,60,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,426.114929,25.231638,99.0,25.168697,NaN,353799.0,w5rkeg,0,"[1,5,1]"
17,md12pcjd,3,True,30,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,217.389618,28.126810,99.0,28.016939,NaN,353799.0,xxd4sh,0,"[1,5,1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2rs65sx5,3,True,90,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,624.264587,23.592827,99.0,23.636141,NaN,353799.0,h5cfb2,0,"[0,10,10]"
209,1qsbj8t9,5,True,90,NaN,4,256,0.95,dncnn,1,...,1.617621e+09,622.264038,23.619362,99.0,23.579559,NaN,353799.0,lxv3ql,0,"[0,10,10]"
210,3sc1pee9,3,True,60,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,435.218781,25.138136,99.0,25.109344,NaN,353799.0,vjw7nm,0,"[0,10,10]"
211,1u9bp41f,7,True,60,NaN,4,256,0.95,dncnn,1,...,1.617623e+09,432.140564,25.169771,99.0,25.076212,NaN,353799.0,l7widm,0,"[0,10,10]"


In [20]:
df_synthetic_denoising_compact_unique_selfonn.groupby([
    'max_shifts',
    'num_neurons',
    'q'
]).count().top_val_psnr

max_shifts  num_neurons  q
[0,0,0]     256          3    2
                         5    3
                         7    2
[0,10,0]    256          3    3
                         5    3
                         7    3
[0,10,10]   128          3    3
                         5    3
                         7    3
            256          3    3
                         5    3
                         7    3
            512          3    3
                         5    3
                         7    3
[0,10,4]    256          3    3
                         5    3
                         7    3
[0,10,8]    256          3    3
                         5    3
                         7    3
[0,15,0]    256          3    3
                         5    3
                         7    3
[0,20,0]    256          3    3
                         5    3
                         7    3
[0,5,0]     256          3    3
                         5    3
                         7    3
            5

In [16]:
# keep only super-onn networks
df_synthetic_denoising_compact_unique_superonn = df_synthetic_denoising_compact_unique_selfonn.iloc[
    np.where(df_synthetic_denoising_compact_unique_selfonn.max_shifts != '[0,0,0]')
]
df_synthetic_denoising_compact_unique_superonn

,ID,q,clip,sigma,model_name,num_layers,num_neurons,train_ratio,dataset_name,num_channels,...,_timestamp,train_loss,top_val_psnr,epoch,CBSD68,trainer/global_step,global_step,Run Name,Version,max_shifts
13,1rjr0f27,7,True,30,NaN,4,512,0.95,dncnn,1,...,1.620126e+09,210.293518,28.278606,99.0,28.066206,NaN,353799.0,bmnviu,0,"[1,5,1]"
14,tnjjfjpn,5,True,30,NaN,4,512,0.95,dncnn,1,...,1.620120e+09,211.740295,28.240484,99.0,28.103859,NaN,353799.0,wyjs64,0,"[1,5,1]"
15,3p3llvdc,3,True,90,NaN,4,512,0.95,dncnn,1,...,1.620111e+09,630.040283,23.523420,99.0,23.586006,NaN,353799.0,4m96ac,0,"[1,5,1]"
16,1vnzy85r,3,True,60,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,426.114929,25.231638,99.0,25.168697,NaN,353799.0,w5rkeg,0,"[1,5,1]"
17,md12pcjd,3,True,30,NaN,4,512,0.95,dncnn,1,...,1.620110e+09,217.389618,28.126810,99.0,28.016939,NaN,353799.0,xxd4sh,0,"[1,5,1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2rs65sx5,3,True,90,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,624.264587,23.592827,99.0,23.636141,NaN,353799.0,h5cfb2,0,"[0,10,10]"
209,1qsbj8t9,5,True,90,NaN,4,256,0.95,dncnn,1,...,1.617621e+09,622.264038,23.619362,99.0,23.579559,NaN,353799.0,lxv3ql,0,"[0,10,10]"
210,3sc1pee9,3,True,60,NaN,4,256,0.95,dncnn,1,...,1.617618e+09,435.218781,25.138136,99.0,25.109344,NaN,353799.0,vjw7nm,0,"[0,10,10]"
211,1u9bp41f,7,True,60,NaN,4,256,0.95,dncnn,1,...,1.617623e+09,432.140564,25.169771,99.0,25.076212,NaN,353799.0,l7widm,0,"[0,10,10]"


In [17]:
df_synthetic_denoising_compact_unique_superonn.groupby([
    'max_shifts',
    'num_neurons',
    'sigma'
]).count().top_val_psnr

max_shifts  num_neurons  sigma
[0,10,0]    256          30       3
                         60       3
                         90       3
[0,10,10]   128          30       3
                         60       3
                         90       3
            256          30       3
                         60       3
                         90       3
            512          30       3
                         60       3
                         90       3
[0,10,4]    256          30       3
                         60       3
                         90       3
[0,10,8]    256          30       3
                         60       3
                         90       3
[0,15,0]    256          30       3
                         60       3
                         90       3
[0,20,0]    256          30       3
                         60       3
                         90       3
[0,5,0]     256          30       3
                         60       3
                         90      

In [18]:
# save dataframes
df_synthetic_denoising_compact_unique_superonn.to_pickle('df_superonns.pkl')

In [18]:
df_rw_denoising

,ID,q,clip,sigma,max_shifts,model_name,num_layers,num_neurons,train_ratio,dataset_name,...,top_val_psnr,Kodak,_step,epoch,_runtime,_timestamp,trainer/global_step,global_step,Run Name,Version
0,4mqdhybd,1,True,0,"[0,0,0]",ridnet,0,0,0.95,sidd,...,41.284206,31.107117,121599.0,99.0,129369.0,1.620599e+09,NaN,NaN,9dcfjy,0
1,3srmqe1b,1,True,0,"[0,0,0]",ridnet,0,0,0.95,sidd,...,41.304935,29.892746,121599.0,99.0,181342.0,1.620463e+09,NaN,NaN,jdgea6,0
2,1brdc1mg,1,True,0,"[0,0,0]",NaN,17,1024,0.95,sidd,...,34.058853,NaN,199.0,99.0,46024.0,1.620308e+09,60799.0,NaN,7wrwdv,0
4,26i8ov3y,7,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,37.980061,26.891420,200.0,99.0,46397.0,1.620305e+09,NaN,60799.0,whzdfg,0
5,26pemhmz,5,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.191158,27.469292,200.0,99.0,35008.0,1.620292e+09,NaN,60799.0,zupt7e,0
6,2vdc69fd,3,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.213318,27.324059,200.0,99.0,23109.0,1.620280e+09,NaN,60799.0,2x24ee,0
7,4t66hzg9,1,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,37.664948,26.866209,200.0,99.0,8946.0,1.620265e+09,NaN,60799.0,cks8by,0
8,tj3r26tc,5,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,39.171474,27.506453,200.0,99.0,34479.0,1.620266e+09,NaN,60799.0,nnoilc,0
9,2fdfpqk8,7,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.869034,27.376518,200.0,99.0,46132.0,1.620278e+09,NaN,60799.0,qlxvxo,0
10,21szvtns,3,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,39.192051,27.736574,200.0,99.0,22909.0,1.620235e+09,NaN,60799.0,j8tmik,0


In [35]:
df_rw_denoising_compact = df_rw_denoising.loc[df_rw_denoising.num_layers==4]
df_rw_denoising_compact

,ID,q,clip,sigma,max_shifts,model_name,num_layers,num_neurons,train_ratio,dataset_name,...,top_val_psnr,Kodak,_step,epoch,_runtime,_timestamp,trainer/global_step,global_step,Run Name,Version
4,26i8ov3y,7,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,37.980061,26.891420,200.0,99.0,46397.0,1.620305e+09,NaN,60799.0,whzdfg,0
5,26pemhmz,5,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.191158,27.469292,200.0,99.0,35008.0,1.620292e+09,NaN,60799.0,zupt7e,0
6,2vdc69fd,3,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.213318,27.324059,200.0,99.0,23109.0,1.620280e+09,NaN,60799.0,2x24ee,0
7,4t66hzg9,1,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,37.664948,26.866209,200.0,99.0,8946.0,1.620265e+09,NaN,60799.0,cks8by,0
8,tj3r26tc,5,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,39.171474,27.506453,200.0,99.0,34479.0,1.620266e+09,NaN,60799.0,nnoilc,0
9,2fdfpqk8,7,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.869034,27.376518,200.0,99.0,46132.0,1.620278e+09,NaN,60799.0,qlxvxo,0
10,21szvtns,3,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,39.192051,27.736574,200.0,99.0,22909.0,1.620235e+09,NaN,60799.0,j8tmik,0
11,2ufjajpa,1,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.523350,26.707682,200.0,99.0,8939.0,1.620220e+09,NaN,60799.0,e0m1jh,0


In [36]:
# download all checkpoints
# Initiate connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('puhti.csc.fi', username="malik", password="TuniPassword2021")
sftp = ssh.open_sftp()
print('Connection open..')

# Download checkpoints
for index,row in df_rw_denoising_compact.iterrows(): get_checkpoint_from_remote(sftp,row.ID)

# Close connection
sftp.close()
ssh.close()
print('Connection closed..')

Connection open..
Checkpoint bm3d_vs_superonn/26i8ov3y/checkpoints/epoch=77.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/26pemhmz/checkpoints/epoch=86.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/2vdc69fd/checkpoints/epoch=89.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/4t66hzg9/checkpoints/epoch=91.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/tj3r26tc/checkpoints/epoch=85.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/2fdfpqk8/checkpoints/epoch=87.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/21szvtns/checkpoints/epoch=99.ckpt already exists. Skipping..
Checkpoint bm3d_vs_superonn/2ufjajpa/checkpoints/epoch=84.ckpt already exists. Skipping..
Connection closed..


In [42]:
import glob
import utils

df_rw_denoising_compact.loc[:,'max_shifts'] = np.nan

# load checkpoints and update max_shifts
ckpt_path = r"{project}//{id}//checkpoints//*.ckpt"
for index,row in df_rw_denoising_compact.iterrows():
    # format path for checkpoint
    ckpt_path_now = glob.glob( ckpt_path.format(project=PROJECT_NAME,id=row.ID) )[0]
    
    # load checkpoint
    model = utils.models.get_model('superonn')
    dn = utils.denoiser.Denoiser.load_from_checkpoint(ckpt_path_now,model=model,strict=False,map_location='cpu')

    # get shifts from model
    model_now = dn.model.model
    assert model_now[2].max_shift == model_now[4].max_shift, "Shifts do not match"
    df_rw_denoising_compact.loc[df_rw_denoising_compact.ID==row.ID,'max_shifts'] = f"[{model_now[0].max_shift},{model_now[2].max_shift},{model_now[6].max_shift}]"


C:\Users\malik\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [43]:
df_rw_denoising_compact

,ID,q,clip,sigma,max_shifts,model_name,num_layers,num_neurons,train_ratio,dataset_name,...,top_val_psnr,Kodak,_step,epoch,_runtime,_timestamp,trainer/global_step,global_step,Run Name,Version
4,26i8ov3y,7,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,37.980061,26.891420,200.0,99.0,46397.0,1.620305e+09,NaN,60799.0,whzdfg,0
5,26pemhmz,5,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.191158,27.469292,200.0,99.0,35008.0,1.620292e+09,NaN,60799.0,zupt7e,0
6,2vdc69fd,3,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,38.213318,27.324059,200.0,99.0,23109.0,1.620280e+09,NaN,60799.0,2x24ee,0
7,4t66hzg9,1,True,0,"[0,0,0]",NaN,4,256,0.95,sidd,...,37.664948,26.866209,200.0,99.0,8946.0,1.620265e+09,NaN,60799.0,cks8by,0
8,tj3r26tc,5,True,0,"[0,5,0]",NaN,4,256,0.95,sidd,...,39.171474,27.506453,200.0,99.0,34479.0,1.620266e+09,NaN,60799.0,nnoilc,0
9,2fdfpqk8,7,True,0,"[0,5,0]",NaN,4,256,0.95,sidd,...,38.869034,27.376518,200.0,99.0,46132.0,1.620278e+09,NaN,60799.0,qlxvxo,0
10,21szvtns,3,True,0,"[0,5,0]",NaN,4,256,0.95,sidd,...,39.192051,27.736574,200.0,99.0,22909.0,1.620235e+09,NaN,60799.0,j8tmik,0
11,2ufjajpa,1,True,0,"[0,5,0]",NaN,4,256,0.95,sidd,...,38.523350,26.707682,200.0,99.0,8939.0,1.620220e+09,NaN,60799.0,e0m1jh,0


In [44]:
df_rw_denoising_compact.to_pickle('df_rw_compact.pkl')